In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torchvision.utils as vutils




In [2]:
% load_ext autoreload 
% autoreload 2

In [3]:
from CVAEWGAN import Encode
from CVAEWGAN import Decode
from CVAEWGAN import NetD

In [4]:
bsz = 100
criterion = nn.MSELoss()


In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=bsz, shuffle=True)


In [6]:
netEnc = Encode()
netDec = Decode()

In [7]:
netEnc = Encode()
netDec = Decode()
netD = NetD(10,20,40)
#aux = Aux()
print(netEnc)

Encode(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (fc21): Linear(in_features=400, out_features=20, bias=True)
  (fc22): Linear(in_features=400, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=400, bias=True)
  (fc4): Linear(in_features=400, out_features=784, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (mu_): Sequential(
    (0): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(8, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 20, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (7): ReLU(inplace)
  )
  (dec_): Sequential(
    (0): ConvTranspose2d(30, 160, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=T

In [8]:
optimizerEnc = optim.Adam(netEnc.parameters(), lr=1e-3)
optimizerDec = optim.Adam(netDec.parameters(), lr=1e-3)
optimizerD = optim.Adam(netD.parameters(), lr=1e-3)

#optimizer_aux = optim.Adam(aux.parameters(), lr=1e-4)

In [9]:
input = torch.FloatTensor(bsz,28,28)
label = torch.FloatTensor(bsz)
real_label=1
fake_label=0
USE_CUDA=1
lamb = 1e-2
l1dist = nn.PairwiseDistance(1)
l2dist = nn.PairwiseDistance(2)
LeakyReLU = nn.LeakyReLU(0)

if(USE_CUDA):
    netEnc=netEnc.cuda()
    netDec=netDec.cuda()
    netD=netD.cuda()
    #aux = aux.cuda()
    #criterion=criterion.cuda()
    input,label=input.cuda(), label.cuda()
    LeakyReLU = LeakyReLU.cuda()

In [10]:
from train import run_trainer

In [12]:
run_trainer(netEnc,netDec,netD,optimizerEnc,optimizerDec,optimizerD,train_loader,bsz)

z.size() torch.Size([100, 20, 1, 1])
label.size() torch.Size([100, 10])
z.size() torch.Size([100, 20])
z.size() torch.Size([100, 20, 1, 1])
label.size() torch.Size([100, 10])
z.size() torch.Size([100, 20])
output_real.mean() tensor(0.4894, device='cuda:0')
output_fake.mean() tensor(0.4876, device='cuda:0')


NotImplementedError: 

In [ ]:
label_ = torch.LongTensor(64,1)

for i in range(64):
    label_[i][0] = 5

onehot_tmp = torch.FloatTensor(64,10)

sample = Variable(torch.randn(64, 20))
onehot_tmp.zero_()
onehot_tmp.scatter_(1,label_,1)
onehot = Variable(onehot_tmp)
    
    
if USE_CUDA==1:
    sample = sample.cuda()
    onehot = onehot.cuda()
    sample = netG.decode_new(torch.cat((sample,onehot),1)).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
                'sample'+ '.png')